# Setting up and running absolute binding free energy calculations

This tutorial gives a step-by-step process to set up an absolute binding free energy (ABFE) simulation campaign using OpenFE. In this tutorial we are performing an absolute binding free energy calculation of toluene binding to T4 Lysozyme.

The absolute binding free energy is obtained through a thermodynamic cycle. The interactions of the molecule are decoupled both in solvent, giving $\Delta G$(solvent) and in the complex, giving $\Delta G$(complex), which allows calculation of the absolute binding free energy, $\Delta G$(ABFE). 

Note: In this `Protocol`, the coulombic interactions of the molecule are fully turned off (annihilated), while the Lennard-Jones interactions are decoupled, meaning the intermolecular interactions are turned off, whilst keeping the intramolecular Lennard-Jones interactions.

## <img src="abfe-cycle.png" alt="Drawing" style="width: 400px;"/>

In [1]:
%matplotlib inline
import openfe

## 1. Loading the ligand

First we must load the chemical models for which we wish to calculate the binding free energy.
In this example these are initially stored in a molfile (`.sdf`).
This can be loaded using the `SDMolSupplier` class from rdkit and passed to openfe.

In [2]:
from rdkit import Chem
supp = Chem.SDMolSupplier("../cookbook/assets/toluene.sdf", removeHs=False)
ligands = [openfe.SmallMoleculeComponent.from_rdkit(mol) for mol in supp]

## 2. Charging the ligand

It is recommended to use a single set of charges for each ligand to ensure reproducibility between repeats. 

Here we will use some utility functions from OpenFE which can assign partial charges to a series of molecules with a variety of methods which can be configured via the `OpenFFPartialChargeSettings` class. In this example 
we will charge the ligands using the `am1bcc` method from `ambertools` which is the default charge scheme used by OpenFE.

In [3]:
from openfe.protocols.openmm_utils.omm_settings import OpenFFPartialChargeSettings
from openfe.protocols.openmm_utils.charge_generation import bulk_assign_partial_charges

charge_settings = OpenFFPartialChargeSettings(partial_charge_method="am1bcc", off_toolkit_backend="ambertools")

ligands = bulk_assign_partial_charges(
    molecules=ligands,
    overwrite=False,  
    method=charge_settings.partial_charge_method,
    toolkit_backend=charge_settings.off_toolkit_backend,
    generate_n_conformers=charge_settings.number_of_conformers,
    nagl_model=charge_settings.nagl_model,
    processors=1
)

/Users/hannahbaumann/miniforge3/envs/openfe_dev/lib/python3.12/site-packages/gufe/components/explicitmoleculecomponent.py:93: UserWarning: Partial charges have been provided, these will preferentially be used instead of generating new partial charges
  warnings.warn(wmsg)
Generating charges: 100%|█████████████████████████| 1/1 [00:00<00:00,  1.79it/s]


## 3. Creating `ChemicalSystem`s

OpenFE describes complex molecular systems as being composed of `Component`s. For example, we have a `SmallMoleculeComponent` for each small molecule, a `SolventComponent` to describe the solvent, and a `ProteinComponent` to describe the protein.

The `Component`s are joined in a `ChemicalSystem` to define the entire system.

In state A of the binding free energy `Protocol`, the ligand is fully interacting in the complex and the `ChemicalSystem` contains the ligand, the protein, and the solvent. In the other endstate, state B, the ligand is fully decoupled in the complex. Therefore, the `ChemicalSystem` in state B only contains the protein and the solvent.

Note that for ABFE simulations, we are not separately defining the solvent state, but the `Protocol` creates that based on the complex states.

In [4]:
# defaults are water with NaCl at 0.15 M
solvent = openfe.SolventComponent()

In [5]:
protein = openfe.ProteinComponent.from_pdb_file("../cookbook/assets/t4_lysozyme.pdb")

In [6]:
# In state A the ligand is fully interacting in the solvent
systemA = openfe.ChemicalSystem({
    'ligand': ligands[0],
    'protein': protein,
    'solvent': solvent,
}, name=ligands[0].name)
# In state B the ligand is fully decoupled in the solvent, therefore we are only defining the solvent here
systemB = openfe.ChemicalSystem({
    'protein': protein,
    'solvent': solvent,
})

## 4. Defining the ABFE simulation settings and creating a `Protocol`

There are various different parameters which can be set to determine how the ABFE simulation will take place. 

The easiest way to customize `Protocol` settings is to start with the default settings, and modify them. Many settings carry units with them.

In [7]:
from openfe.protocols.openmm_afe import AbsoluteBindingProtocol

/Users/hannahbaumann/miniforge3/envs/openfe_dev/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [8]:
settings = AbsoluteBindingProtocol.default_settings()

In [9]:
settings

{'alchemical_settings': {},
 'complex_equil_output_settings': {'checkpoint_interval': <Quantity(250.0, 'picosecond')>,
                                   'checkpoint_storage_filename': 'checkpoint.chk',
                                   'equil_npt_structure': 'equil_npt_structure.pdb',
                                   'equil_nvt_structure': 'equil_nvt_structure.pdb',
                                   'forcefield_cache': 'db.json',
                                   'log_output': 'equil_simulation.log',
                                   'minimized_structure': 'minimized.pdb',
                                   'output_indices': 'all',
                                   'preminimized_structure': 'system.pdb',
                                   'production_trajectory_filename': 'production_equil.xtc',
                                   'trajectory_write_interval': <Quantity(20.0, 'picosecond')>},
 'complex_equil_simulation_settings': {'equilibration_length': <Quantity(0.5, 'nanosecon

By default we run 3 repeats with solvent and complex simulation lengths of 10 ns over 14 and 30 lambda windows. 
To speed things up here we instead run 1 repeat with both solvent and complex simulation lengths of 0.5 ns over 14 and 30 lambda windows.

Changing default values:

In [9]:
from openff.units import unit

# change the values
settings.protocol_repeats = 1
settings.solvent_simulation_settings.equilibration_length = 10 * unit.picosecond
settings.solvent_simulation_settings.production_length = 500 * unit.picosecond
settings.complex_simulation_settings.equilibration_length = 10 * unit.picosecond
settings.complex_simulation_settings.production_length = 500 * unit.picosecond
settings.engine_settings.compute_platform = 'CUDA'

### Creating the `Protocol`
With the Settings inspected and adjusted, we can provide these to the `Protocol`. This `Protocol` defines the procedure to estimate a free energy difference between two chemical systems, with the details of the two end states yet to be defined.

In [10]:
protocol = AbsoluteBindingProtocol(settings=settings)

## 5. Running the ABFE simulation

### (a) Using the CLI

Once we have the ChemicalSystems, and the `Protocol`, we can create the Transformation. 

In [11]:
transformation = openfe.Transformation(
            stateA=systemA,
            stateB=systemB,
            mapping=None,
            protocol=protocol,  # use protocol created above
            name=f"{systemA.name}"
        )

We'll write out the transformation to disk, so that it can be run using the openfe quickrun command:

In [12]:
import pathlib
# first we create the directory
transformation_dir = pathlib.Path("abfe_json")
transformation_dir.mkdir(exist_ok=True)

# then we write out the transformation
transformation.dump(transformation_dir / f"{transformation.name}.json")

You can run the ABFE simulation from the CLI by using the `openfe quickrun` command. It takes a transformation JSON as input, and the flags -o to give the final output JSON file and -d for the directory where simulation results should be stored. For example,

`openfe quickrun path/to/transformation.json -o results.json -d working-directory`

where path/to/transformation.json is the path to one of the files created above.

### (b) Using the Python API

**Creating the `ProtocolDAG`**

Once we have the two `ChemicalSystem`s, and the `Protocol`, we can create the `ProtocolDAG`.

This creates a directed-acyclic-graph (DAG) of computational tasks necessary for creating an estimate of the free energy difference between the two chemical systems.

In [13]:
dag = protocol.create(stateA=systemA, stateB=systemB, mapping=None)

To summarize, this `ProtocolDAG` contains:
- chemical models of both sides of the alchemical transformation in `systemA` and `systemB`
- a description of the exact computational algorithm to use to perform the estimate in `Protocol`
- the `mapping` is set to `None` since no atoms are mapped in the ABFE protocol

**Executing the simulation**

The DAG contains many invdividual jobs. We could execute them sequentially in this notebook using the `gufe.protocols.execute` function, however, this calculation is too expensive to be run in that manner, therefore the `execulte_DAG` command is commented out here.

In a more realistic (expansive) situation we would farm off the individual jobs to a HPC cluster or cloud compute service so they could be executed in parallel.

Note: we use the `shared_basedir` and `scratch_basedir` argument of `execute_DAG` in order to set the directory where the simulation files are written to

In [14]:
from gufe.protocols import execute_DAG
import pathlib

In [16]:
# Finally we can run the simulations
path = pathlib.Path('./abfe_results')
path.mkdir()

# Execute the DAG
# Commented out since this would be too expensive to run in this notebook.
# dag_results = execute_DAG(dag, scratch_basedir=path, shared_basedir=path, n_retries=3)

## 6. Analysis

Finally now that we've run our simulations, let's go ahead and gather the free
energies for both phases.

### Python API

If you executed the simulations using the Python API, you will have generated a `dag_results` object. You can analyze these results by calling the `Protocol`s' `gather()` method. This takes a **list** of completed DAG results and returns a `AbsoluteBindingProtocolResult` which can return a free energy estimate and uncertainty by calling the `get_estimate()` and `get_uncertainty()` methods.

The code for this part is commented out since executing the DAG in this notebook would be too expensive. 

In [17]:
# Get the complex and solvent results
# protocol_results = protocol.gather([dag_results])

# print(f"ABFE dG: {protocol_results.get_estimate()}, err {protocol_results.get_uncertainty()}")

You can save the `AbsoluteBindingProtocolResult` to a JSON output file in the following manner:

In [18]:
# Save the results in a json file
# import gzip
# import json
# import gufe
# outdict = {
#     "estimate": protocol_results.get_estimate(),
#     "uncertainty": protocol_results.get_uncertainty(),
#     "protocol_result": protocol_results.to_dict(),
#     "unit_results": {
#         unit.key: unit.to_keyed_dict()
#         for unit in dag_results.protocol_unit_results
#     }
# }

# with open("abfe_json/toluene_results.json") as stream:
#     json.dump(outdict, stream, cls=gufe.tokenization.JSON_HANDLER.encoder)

### CLI / Quickrun

If you ran the simulation using the CLI (i.e. by calling `openfe quickrun` ) you will end up with the same JSON output file as the one created in the previous cell. To get your simulation results you can load them back into Python in the following manner:

In [12]:
import gzip
import json
import gufe

outfile = "abfe_results/toluene_results.json"
with open(outfile) as stream:
    results = json.load(stream)
    estimate = results['estimate']
    uncertainty = results['uncertainty']

In [13]:
estimate

{'magnitude': -4.671169105743085,
 'unit': 'kilocalorie_per_mole',
 ':is_custom:': True,
 'pint_unit_registry': 'openff_units'}